In [53]:
from plotly_calplot import calplot
import pandas as pd
import nbformat
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

# リサーチマップのアクセス解析

In [54]:
df = pd.read_csv("access_data/researchmap.csv")
df['date'] = pd.to_datetime(df['date'])
df.columns

Index(['date', 'value'], dtype='object')

In [55]:
custom_colorscale = [
    [0, "#3f3f3f"],  # スケールの最小値に対応する色
    [1, "#00d5ff"]    # スケールの最大値に対応する色
]

# タイルプロット
fig = calplot(df, x="date", y="value",
              colorscale=custom_colorscale)
fig.update_layout({"yaxis": {"title": "2024"},
                  "plot_bgcolor": "#3f3f3f",
                  "paper_bgcolor": "#3f3f3f"})
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
fig.write_html("html_figures/researchmap.html")
fig

# 個人ページのアクセス解析

In [56]:
df = pd.read_csv("access_data/personal.csv")
df['date'] = pd.to_datetime(df['date'])
df.columns

Index(['date', 'value'], dtype='object')

In [57]:
custom_colorscale = [
    [0, "#3f3f3f"],  # スケールの最小値に対応する色
    [1, "#fff700"],       # 中間値に対応する色
]

# タイルプロット
fig = calplot(df, x="date", y="value",
              colorscale=custom_colorscale)
fig.update_layout({"yaxis": {"title": "2024"},
                  "plot_bgcolor": "#3f3f3f",
                  "paper_bgcolor": "#3f3f3f"})
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
fig.write_html("html_figures/personal.html")
fig

# リサーチマップと個人サイトのアクセス数推移

In [58]:
start_date = '2024-11-26'
end_date = '2025-11-26'

# リサーチマップデータ
df_rm = pd.read_csv("access_data/researchmap.csv")
df_rm['date'] = pd.to_datetime(df_rm['date'])
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
df_date = pd.DataFrame(date_range, columns=['date'])
df_rm = pd.merge(df_date, df_rm, on='date', how='left')
df_rm["category"] = "researchmap"


# このサイトのデータ
df_p = pd.read_csv("access_data/personal.csv")
df_p['date'] = pd.to_datetime(df_p['date'])
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
df_date = pd.DataFrame(date_range, columns=['date'])
df_p = pd.merge(df_date, df_p, on='date', how='left')
df_p["category"] = "this page"

# 2つのでデータを連結
df = pd.concat([df_rm, df_p])
df

,date,value,category
0,2024-11-26,18.0,researchmap
1,2024-11-27,4.0,researchmap
2,2024-11-28,2.0,researchmap
3,2024-11-29,2.0,researchmap
4,2024-11-30,2.0,researchmap
...,...,...,...
361,2025-11-22,NaN,this page
362,2025-11-23,NaN,this page
363,2025-11-24,NaN,this page
364,2025-11-25,NaN,this page


In [59]:
line = px.line(df, x='date', y='value', color = "category",
        color_discrete_map={'researchmap': '#00d5ff', 'this page': '#f2ff00'})
line.update_layout(margin=dict(l=10, r=10, t=10, b=10))
line.update_layout(
    legend=dict(
        x=0,  # x位置（0で左端）
        y=1,  # y位置（1で上端）
        traceorder='normal',  # 凡例アイテムの順番
        orientation='v'  # 縦方向で配置
    )
)
line.update_layout(
    yaxis=dict(
        showgrid=True,  # y軸の目盛り線を表示
        showticklabels=True,  # y軸の目盛りラベルを表示
    )
)
line.update_layout(
    plot_bgcolor="white",  # 描画領域の背景色を白
    paper_bgcolor="white",  # グラフ全体の背景色を白
    xaxis=dict(
        showgrid=True,  # 目盛り線を表示
        gridcolor="gray",  # x軸の目盛り線をグレーに設定
    ),
    yaxis=dict(
        showgrid=True,  # 目盛り線を表示
        gridcolor="gray",  # y軸の目盛り線をグレーに設定
    ),
    legend=dict(
        x=0,
        y=1,
        traceorder='normal',
        orientation='v'
    )
)

line.update_layout({"plot_bgcolor": "#3f3f3f",
                    "paper_bgcolor": "#3f3f3f"})

line.update_layout(
    xaxis_title='',  # x軸のラベルを空に
    yaxis_title=''   # y軸のラベルを空に
)

line

In [60]:
line.write_html("html_figures/lineplot.html")

# 地図を描こう
### アクセスされているけど，まだ緯度経度を登録していない地域を検出

In [61]:
rm_region = pd.read_csv("access_data/researchmap_region.csv")
p_region = pd.read_csv("access_data/personal_region.csv")
region = pd.concat([rm_region, p_region]).groupby(["地域", "市区町村"]).sum().reset_index()
region_pref_muni = region[["地域", "市区町村"]].rename(columns = {"地域":"pref", "市区町村":"city"})
#region_pref_muni.to_csv("access_data/region_pref_muni.csv")
region_pref_muni

,pref,city
0,(not set),(not set)
1,Aichi,Kasugai
2,Aichi,Nagoya
3,Aichi,Toyohashi
4,Akita,Akita
...,...,...
134,Toyama,Nanto
135,Virginia,Ashburn
136,Yamagata,Yamagata
137,Yamanashi,Kofu


In [62]:
coords = pd.read_csv("access_data/lon_lat.csv")
coords_region = pd.merge(region_pref_muni, coords, how="outer")
coords_region = coords_region[coords_region["lon"].isna() == True]
coords_region[["pref", "city"]].to_csv("access_data/dont_have_lon_lat.csv")
# まだ座標を当たられていない場所を取得

### リサーチマップ

In [63]:
# 座標の読み込み
coords = pd.read_csv("access_data/lon_lat.csv")

# rmアクセス数の読み込み
rm_region = pd.read_csv("access_data/researchmap_region.csv")
rm_region["city"] = rm_region["市区町村"]
rm_region["active_user"] = rm_region["アクティブ ユーザー"]
rm_df = rm_region.merge(coords)
rm_df["cite"] = "researchmap"
rm_df

,地域,市区町村,アクティブ ユーザー,新規ユーザー数,エンゲージのあったセッション数,エンゲージメント率,エンゲージメントのあったセッション数（1 アクティブ ユーザーあたり）,アクティブ ユーザーあたりの平均エンゲージメント時間,イベント数,キーイベント,ユーザー キーイベント率,合計収益,city,active_user,pref,lat,lon,cite
0,Tokyo,(not set),314,311,166,0.498498,0.528662,18.726115,1311,0,0,0,(not set),314,(not set),35.680929,139.763317,researchmap
1,Tokyo,(not set),314,311,166,0.498498,0.528662,18.726115,1311,0,0,0,(not set),314,Tokyo,NaN,NaN,researchmap
2,Tokyo,(not set),314,311,166,0.498498,0.528662,18.726115,1311,0,0,0,(not set),314,Colorado,39.734654,-105.050449,researchmap
3,Tokyo,(not set),314,311,166,0.498498,0.528662,18.726115,1311,0,0,0,(not set),314,Ontario,43.661520,-79.387780,researchmap
4,Tokyo,(not set),314,311,166,0.498498,0.528662,18.726115,1311,0,0,0,(not set),314,Taipei City,25.033059,121.564738,researchmap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,Tokyo,Shinagawa City,1,1,0,0.000000,0.000000,0.000000,3,0,0,0,Shinagawa City,1,Tokyo,35.608923,139.730630,researchmap
142,Toyama,Nanto,1,1,0,0.000000,0.000000,0.000000,3,0,0,0,Nanto,1,Toyama,36.511040,136.899933,researchmap
143,Yamagata,Yamagata,1,1,1,1.000000,1.000000,11.000000,4,0,0,0,Yamagata,1,Yamagata,38.248281,140.327441,researchmap
144,Zurich,Zurich,1,1,0,0.000000,0.000000,0.000000,3,0,0,0,Zurich,1,Zurich,47.377948,8.536401,researchmap


### 個人サイト

In [64]:
# 座標の読み込み
coords = pd.read_csv("access_data/lon_lat.csv")

# rmアクセス数の読み込み
p_region = pd.read_csv("access_data/personal_region.csv")
p_region["city"] = p_region["市区町村"]
p_region["active_user"] = p_region["アクティブ ユーザー"]
p_df = p_region.merge(coords)
p_df["cite"] = "personal"
p_df

,地域,市区町村,アクティブ ユーザー,新規ユーザー数,エンゲージのあったセッション数,エンゲージメント率,エンゲージメントのあったセッション数（1 アクティブ ユーザーあたり）,アクティブ ユーザーあたりの平均エンゲージメント時間,イベント数,キーイベント,ユーザー キーイベント率,合計収益,city,active_user,pref,lat,lon,cite
0,Tokyo,(not set),41,39,36,0.545455,0.878049,15.951220,221,0,0,0,(not set),41,(not set),35.680929,139.763317,personal
1,Tokyo,(not set),41,39,36,0.545455,0.878049,15.951220,221,0,0,0,(not set),41,Tokyo,NaN,NaN,personal
2,Tokyo,(not set),41,39,36,0.545455,0.878049,15.951220,221,0,0,0,(not set),41,Colorado,39.734654,-105.050449,personal
3,Tokyo,(not set),41,39,36,0.545455,0.878049,15.951220,221,0,0,0,(not set),41,Ontario,43.661520,-79.387780,personal
4,Tokyo,(not set),41,39,36,0.545455,0.878049,15.951220,221,0,0,0,(not set),41,Taipei City,25.033059,121.564738,personal
5,Miyagi,Sendai,18,17,145,0.763158,8.055556,239.833333,997,0,0,0,Sendai,18,Miyagi,38.268050,140.872263,personal
6,Colorado,(not set),10,10,0,0.000000,0.000000,0.000000,30,0,0,0,(not set),10,(not set),35.680929,139.763317,personal
7,Colorado,(not set),10,10,0,0.000000,0.000000,0.000000,30,0,0,0,(not set),10,Tokyo,NaN,NaN,personal
8,Colorado,(not set),10,10,0,0.000000,0.000000,0.000000,30,0,0,0,(not set),10,Colorado,39.734654,-105.050449,personal
9,Colorado,(not set),10,10,0,0.000000,0.000000,0.000000,30,0,0,0,(not set),10,Ontario,43.661520,-79.387780,personal


In [65]:
df = pd.concat([rm_df, p_df])
df

,地域,市区町村,アクティブ ユーザー,新規ユーザー数,エンゲージのあったセッション数,エンゲージメント率,エンゲージメントのあったセッション数（1 アクティブ ユーザーあたり）,アクティブ ユーザーあたりの平均エンゲージメント時間,イベント数,キーイベント,ユーザー キーイベント率,合計収益,city,active_user,pref,lat,lon,cite
0,Tokyo,(not set),314,311,166,0.498498,0.528662,18.726115,1311,0,0,0,(not set),314,(not set),35.680929,139.763317,researchmap
1,Tokyo,(not set),314,311,166,0.498498,0.528662,18.726115,1311,0,0,0,(not set),314,Tokyo,NaN,NaN,researchmap
2,Tokyo,(not set),314,311,166,0.498498,0.528662,18.726115,1311,0,0,0,(not set),314,Colorado,39.734654,-105.050449,researchmap
3,Tokyo,(not set),314,311,166,0.498498,0.528662,18.726115,1311,0,0,0,(not set),314,Ontario,43.661520,-79.387780,researchmap
4,Tokyo,(not set),314,311,166,0.498498,0.528662,18.726115,1311,0,0,0,(not set),314,Taipei City,25.033059,121.564738,researchmap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,Tokyo,Nerima City,1,1,1,1.000000,1.000000,9.000000,4,0,0,0,Nerima City,1,Tokyo,35.746888,139.617910,personal
50,Tokyo,Shinjuku City,1,1,1,1.000000,1.000000,7.000000,4,0,0,0,Shinjuku City,1,Tokyo,35.693776,139.703742,personal
51,Yamanashi,Kofu,1,1,0,0.000000,0.000000,0.000000,3,0,0,0,Kofu,1,Yamanashi,35.660131,138.575896,personal
52,Aichi,Nagoya,0,0,0,0.000000,0.000000,0.000000,2,0,0,0,Nagoya,0,Aichi,35.171006,136.884076,personal


In [66]:
fig = px.scatter_mapbox(
    df,
    lat='lat',  # 緯度
    lon='lon',  # 経度
    size='active_user',
    center={'lat': 37.076045, 'lon': 136.971889},
    zoom=4,
    size_max=40,
    color_discrete_sequence=["#00d5ff", "#fff700"],
    color="cite"  # サイト別に色分け
)

fig.update_layout(
    title_font_size=20,
    geo=dict(
        showcountries=True,  # 国境を表示
        showcoastlines=True,  # 海岸線を表示
        coastlinecolor="Gray",
        landcolor="LightYellow"
    ),
    mapbox_style='carto-darkmatter',
    margin=dict(l=0, r=0, t=0, b=0),
    legend=dict(
        x=1,  # 凡例を右に移動
        y=1,  # 凡例を上に移動
        xanchor="right",  # 凡例の右端を基準に配置
        yanchor="top",  # 凡例の上端を基準に配置
        bgcolor="rgba(255, 255, 255, 0.7)",  # 背景を半透明の白に設定
        bordercolor="Black",  # 枠線の色
        borderwidth=1  # 枠線の幅
    )
)

fig.write_html("html_figures/access_map.html")
fig.show()
